In [2]:
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle

from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import fsspec

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12,8

# !pip install opencv-python-headless
import cv2
from matplotlib.patches import Rectangle
from typing import List, Tuple
import itertools
from copy import copy, deepcopy
import os
import shutil

import getpass
import azure.storage.blob
from azure.storage.blob import BlobClient, BlobServiceClient
from azure.core.exceptions import ResourceExistsError


     |████████████████████████████████| 37.1 MB 14.8 MB/s eta 0:00:01     |████████████████▌               | 19.2 MB 16.0 MB/s eta 0:00:02


In [3]:
"""
Complete pipeline to find heat events. Process by year.

Input: the np arrays produced by the 1st process before this pipeline. These are the same size 
        arrays that have a heat event flag per iX,iY. But each location of unaware from each other. This
        pipe aggregates them in x,y axeses first, then in time axis.
Output: 3D Bounding boxes of all heat events.
"""

def bounding_boxes(arr2d: np.array) -> List[tuple]:
    
    H = arr2d.astype(np.uint8)
    ret, thresh = cv2.threshold(H, 0, 1, 0, cv2.THRESH_BINARY)
    contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes = [cv2.boundingRect(c) for c in contours]
    return boxes

def isoverlap(box1:tuple, box2:tuple) -> bool:
    """Return True if two windows overlap"""
    x1,y1,w1,h1 = box1
    x2,y2,w2,h2 = box2
    return not (x2>x1+w1 or x2+w2<x1 or y2>y1+h1 or y2+h2<y1)

def outer(box1:tuple, box2:tuple) -> tuple:
    """Fuse two windows into one, parent window."""
    x1,y1,w1,h1 = box1
    x2,y2,w2,h2 = box2
    x = min(x1,x2)
    y = min(y1,y2)
    w = max(x1+w1,x2+w2)-x
    h = max(y1+h1,y2+h2)-y
    return (x, y, w, h)

def istiny(box:tuple, min_area:int) -> bool:
    x,y,w,h = box
    return w*h <= min_area

def filter_tiny_ones(boxes:List[tuple]) -> List[tuple]:
    return [c for c in boxes if not istiny(c, 10)]

def collapse(boxes:List[tuple]) -> List[tuple]:
    
    for box1, box2 in itertools.combinations(boxes, 2):
        if isoverlap(box1,box2):
            boxes.remove(box1)
            boxes.remove(box2)
            boxes.append(outer(box1,box2))
            return collapse(boxes) # recursion

    boxes.sort(key=lambda _:_[0])
    return boxes

def array2boxes(arr2d:np.array) -> List[tuple]:
    """Pipeline. Takes a time-slice (2D array) and returns the boxes."""
    boxes = bounding_boxes(arr2d)
    boxes = filter_tiny_ones(boxes)
    boxes = collapse(boxes)
    return boxes

def groupby_heat_events(arr3d:np.array) -> List[dict]:
    rows = []
    num_days = arr3d.shape[0]

    for i in range(num_days):
        arr2d = arr3d[i,:,:]
        boxes = array2boxes(arr2d)
        rows += [dict(time=i, boxes=boxes)]
    df = pd.DataFrame(rows)
    df['hasEvent'] = df['boxes'].apply(lambda x: len(x)) > 0
    df['label'] = df['hasEvent'].diff().ne(False).cumsum()

    dff = df[df['hasEvent']]

    dfg = dff.groupby('label').agg({
        'time':[np.min,np.max], 
        'boxes':lambda _: collapse(np.sum(_))
    }).reset_index()
    dfg.columns = ['label', 'i1', 'i2', 'boxes']
    dfg = dfg.assign(d1=dr[dfg['i1']], d2=dr[dfg['i2']])
    dfg = dfg.drop('label', axis=1)
    
    return dfg


# Test it

year = 1971
dr = pd.date_range(start=f'1/1/{year}', periods=365, freq='D').date
arr3d = np.load(f'Koray/CMIP5_flagged/arr_heat3d-{year}.npy')

df_events = groupby_heat_events(arr3d)
df_events

FileNotFoundError: [Errno 2] No such file or directory: 'Koray/CMIP5_flagged/arr_heat3d-1971.npy'

In [421]:
# e.g. there is a heat wave that took 23 days!
np.unique(np.unique(arr3d[np.where(arr3d>0)], return_counts=True)[1], return_counts=True)

(array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 23]),
 array([6544, 2986, 1067,  903,  445,  483,  447,  175,   61,   38,   26,
          16,    7,    7,    6,    6,    1,    1,    1]))

In [ ]:
# Set up Azure access
sas_token = getpass.getpass() 
url_prefix = 'https://nexdcp30.blob.core.windows.net'
blob_folder = "images"

In [422]:
############### INPUTS ######################
lat_min = 0   
lat_max = 50  
lon_min = 220 
lon_max = 300 
############################################# 

area = dict(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))

# temp data
ds = xr.open_mfdataset('Koray/CMIP5/*.nc')

# Ravi's avg 
ds_avg = xr.open_dataset("Koray/Avg_temp_max_CMIP5__30_yrs__1950_to_1979.nc")


for year in range(1970,1980):

    dr = pd.date_range(start=f'1/1/{year}', periods=365, freq='D').date
    arr3d = np.load(f'Koray/CMIP5_flagged/arr_heat3d-{year}.npy') # must be ready
    df_events = groupby_heat_events(arr3d)

    for ev, (i1, i2) in df_events[['i1','i2']].iterrows():
        # to write the image to disk
        folder_name = f"{year}-event{str(ev).zfill(2)}"
        folder_path = f"Koray/CMIP5_images/{folder_name}"
        if not os.path.exists(folder_path):
            os.mkdir(folder_path)
        
        # loop through heat events, some are 3 days long, some are 20
        for i, idx in enumerate(range(i1,i2+1)):

            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,4))

            day = dr[idx].strftime("%Y-%m-%d")
            tasmax = ds['tasmax'].sel(time=day, **area)
            tavg3d = ds_avg['tasmaxavg'].sel(day=idx, **area)
            tdiff = tasmax - tavg3d

            im1 = tdiff.squeeze().plot.imshow(ax=ax1, cmap='Reds', vmin=-4, vmax=4)
            im2 = (tasmax-273.15).squeeze().plot.imshow(ax=ax2, vmin=5, vmax=50)

            # add bounding boxes
            boxes = df_events['boxes'].iloc[ev]
            for b in boxes:
                x,y,w,h = b
                x = float(tdiff.coords['lon'][x])
                y = float(tdiff.coords['lat'][y])
                _, leny, lenx = tdiff.shape

                w = w * (lon_max-lon_min) / lenx
                h = h * (lat_max-lat_min) / leny 

                rect = Rectangle((x, y), w, h, color='b', fill=False, linewidth=2)
                ax1.add_patch(rect)
                rect = Rectangle((x, y), w, h, color='b', fill=False, linewidth=2)
                ax2.add_patch(rect)

            fig.tight_layout()

            # save each image, each event has multiple images, one per event day.
            filename = f"{folder_path}/{day}.png"
            fig.savefig(filename, dpi=fig.dpi)
            plt.close(fig)

            # upload image to azure
            sas_url = f"{url_prefix}/{blob_folder}/{filename}?{sas_token}"
            blob_client = BlobClient.from_blob_url(sas_url)
            with open(filename, "rb") as f:
                try:
                    blob_client.upload_blob(f)
                except ResourceExistsError:
                    pass
    
        # flush VM disk space
        shutil.rmtree(folder_path)
